In [1]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
# Fungsi baru untuk ekstraksi fitur GLCM
def extract_glcm_features(image):
    # Kuantisasi gambar untuk mengurangi level intensitas
    levels = 8
    image_quantized = (image // (256 // levels)).astype('uint8')

    # Definisikan jarak dan sudut
    distances = [1, 2]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°

    # Hitung GLCM
    glcm = graycomatrix(image_quantized, 
                       distances=distances,
                       angles=angles,
                       levels=levels,
                       symmetric=True,
                       normed=True)

    # Ekstrak properti GLCM
    properties = ['homogeneity', 'energy', 'correlation', 'entropy']
    features = []

    for prop in properties:
        feature = graycoprops(glcm, prop)
        features.extend(feature.flatten())  

    return np.array(features)

In [3]:

properties = ['homogeneity', 'energy', 'correlation', 'entropy']
distances = [1, 2]
angles = [0, 45, 90, 135]  

feature_names = []

for prop in properties:
    for d in distances:
        for a in angles:
            feature_names.append(f'{prop}_d{d}_a{a}')

print(feature_names)
print(f"Total kolom: {len(feature_names)}")


['homogeneity_d1_a0', 'homogeneity_d1_a45', 'homogeneity_d1_a90', 'homogeneity_d1_a135', 'homogeneity_d2_a0', 'homogeneity_d2_a45', 'homogeneity_d2_a90', 'homogeneity_d2_a135', 'energy_d1_a0', 'energy_d1_a45', 'energy_d1_a90', 'energy_d1_a135', 'energy_d2_a0', 'energy_d2_a45', 'energy_d2_a90', 'energy_d2_a135', 'correlation_d1_a0', 'correlation_d1_a45', 'correlation_d1_a90', 'correlation_d1_a135', 'correlation_d2_a0', 'correlation_d2_a45', 'correlation_d2_a90', 'correlation_d2_a135', 'entropy_d1_a0', 'entropy_d1_a45', 'entropy_d1_a90', 'entropy_d1_a135', 'entropy_d2_a0', 'entropy_d2_a45', 'entropy_d2_a90', 'entropy_d2_a135']
Total kolom: 32


In [4]:
# Fungsi untuk mendeteksi wajah
def detect_face(img):
    # Load Haar Cascades
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Konversi gambar ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Deteksi wajah
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    # Periksa jika tidak ada wajah yang terdeteksi
    if len(faces) == 0:
        print("No Face Found")
        return None

    # Crop wajah yang terdeteksi dan simpan dalam list
    cropped_faces = []
    for (x, y, w, h) in faces:
        face_crop = gray[y:y + h, x:x + w]
        cropped_faces.append(face_crop)

    return cropped_faces  # Mengembalikan list wajah yang terpotong

In [5]:
# Fungsi utama untuk memproses dataset
def process_dataset(folder_path):
    # List untuk menyimpan hasil
    features_list = []
    file_names = []
    
    # Membaca semua file gambar
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png')):
            # Baca gambar
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            
            if image is not None:
                # Deteksi wajah
                faces = detect_face(image)
                
                if faces:
                    for face in faces:
                        # Resize gambar menjadi 128x128 untuk konsistensi
                        resized_face = cv2.resize(face, (128, 128))
                        
                        # Ekstrak fitur GLCM
                        glcm_features = extract_glcm_features(resized_face)
                        
                        # Simpan hasil
                        features_list.append(glcm_features)
                        file_names.append(filename)
                else:
                    print(f"Tidak ada wajah terdeteksi pada gambar: {image_path}")
            else:
                print(f"Gagal membaca gambar: {image_path}")
    
    return np.array(features_list), file_names

In [6]:
def save_features_to_csv(features, filenames, output_file):
    # Definisikan kolom
    # columns = ['filename'] + [f'feature_{i}' for i in range(features.shape[1])]

    columns = ['filename'] + feature_names
    
    # Buat DataFrame
    data = np.hstack((np.array(filenames).reshape(-1, 1), features))
    print(f"Shape of data: {data.shape}, Shape of columns: {len(columns)}")
    df = pd.DataFrame(data, columns=columns)
    
    # Simpan ke CSV
    df.to_csv(output_file, index=False)

In [7]:
# Main function
def main():
    # Path folder dataset
    train_folder_path = "../env/My Dataset3"
    # test_folder_path = "../env/test"
    
    print("Memulai ekstraksi fitur untuk data latih...")
    
    # Ekstrak fitur dari dataset pelatihan
    train_features, train_filenames = process_dataset(train_folder_path)
    print(f"Jumlah fitur yang diekstrak: {train_features.shape[1]}")
    
    # Simpan hasil untuk data latih
    save_features_to_csv(train_features, train_filenames, 'glcm_features_training_dataset_50_6.csv')
    
    # print("Memulai ekstraksi fitur untuk data test...")
    
    # # Ekstrak fitur dari dataset pengujian
    # test_features, test_filenames = process_dataset(test_folder_path)
    
    # # Simpan hasil untuk data uji
    # save_features_to_csv(test_features, test_filenames, 'glcm_features_test2.csv')

if __name__ == "__main__":
    main()

Memulai ekstraksi fitur untuk data latih...
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (14).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (18).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (37).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (40).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (41).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (42).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (48).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Albert (49).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Alvin (34).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Dataset3\Alvin (35).jpg
No Face Found
Tidak ada wajah terdeteksi pada gambar: ../env/My Data

In [7]:
# Main function
def main():
    # Path folder dataset
    # train_folder_path = "../env/My Dataset"
    test_folder_path = "../env/test2"
    
    # print("Memulai ekstraksi fitur untuk data latih...")
    
    # Ekstrak fitur dari dataset pelatihan
    # train_features, train_filenames = process_dataset(train_folder_path)
    # print(f"Jumlah fitur yang diekstrak: {train_features.shape[1]}")
    
    # Simpan hasil untuk data latih
    # save_features_to_csv(train_features, train_filenames, 'glcm_features_train2.csv')
    
    print("Memulai ekstraksi fitur untuk data test...")
    
    # Ekstrak fitur dari dataset pengujian
    test_features, test_filenames = process_dataset(test_folder_path)
    
    # # Simpan hasil untuk data uji
    save_features_to_csv(test_features, test_filenames, 'glcm_features_testing_new_may.csv')

if __name__ == "__main__":
    main()

Memulai ekstraksi fitur untuk data test...
Shape of data: (11, 33), Shape of columns: 33
